In [ ]:
# Pfizer Assigunment

In [186]:
#Import Pandas
import pandas as pd
import numpy as np

In [187]:
#Import Excel
df_mm = pd.read_excel('C:/Users/varun/OneDrive/Desktop/Pfizer Assignment/Data tables for applicants_Randomized.xlsx',sheet_name='Material Movement')
df_bom = pd.read_excel('C:/Users/varun/OneDrive/Desktop/Pfizer Assignment/Data tables for applicants_Randomized.xlsx',sheet_name='BOM')

In [188]:
df_mm.dtypes

Material Number_R             object
Part Type Code                object
Batch Number_R                object
Movement Code                float64
Movement Indicator            object
Prod Order_R                  object
Posting Date          datetime64[ns]
Plant Code_R                  object
dtype: object

In [189]:
#Create Production Order Start and End Date Table

#Select only relevant columns Prod Order_R and Posting Date
df_mm_prod = df_mm[['Prod Order_R','Posting Date']]

#Remove production orders value equals 'Not Set' and blanks
df_mm_prod = df_mm_prod[df_mm_prod['Prod Order_R'].str.contains("Not Set", na=True)==False]

#Group by to get min and max production order date
df_prod_start_end = df_mm_prod.groupby('Prod Order_R').agg({'Posting Date': ['count', 'min', 'max']}).reset_index()

#Convert multiindex df to single index
df_prod_start_end.columns = [' '.join(col).strip() for col in df_prod_start_end.columns.values]
df_prod_start_end

,Prod Order_R,Posting Date count,Posting Date min,Posting Date max
0,M77565,1,2017-09-19,2017-09-19
1,N99044,1,2017-01-25,2017-01-25
2,R68257,1,2017-03-03,2017-03-03
3,R85963,1,2017-01-12,2017-01-12
4,R87871,1,2017-03-21,2017-03-21
...,...,...,...,...
4040,W67583,2,2018-02-21,2018-02-21
4041,W67787,2,2018-02-22,2018-02-22
4042,W698101,1,2018-03-01,2018-03-01
4043,W698116,2,2018-03-02,2018-03-02


In [190]:
#Create Table to link levels of production orders 

#remove production orders values equal 'Not Set' and blanks
df_prod_ord_link = df_mm[df_mm['Prod Order_R'].str.contains("Not Set", na=True)==False]

# Get indexes where Prod Order_R = Batch Number_R
index_po_batch = df_prod_ord_link[df_prod_ord_link['Prod Order_R'] == df_prod_ord_link['Batch Number_R']].index
# Delete these row indexes from dataFrame
df_prod_ord_link = df_prod_ord_link.drop(index_po_batch)

#Keep only relevant columns and drop duplicate production orders
df_prod_ord_link = df_prod_ord_link[['Prod Order_R','Batch Number_R','Material Number_R']].drop_duplicates(subset=['Prod Order_R'])

#Rename columns
df_prod_ord_link = df_prod_ord_link.rename(columns={"Prod Order_R": "First Level Prod Ord", "Batch Number_R": "Linked Second Level Prod Ord" })
df_prod_ord_link

,First Level Prod Ord,Linked Second Level Prod Ord,Material Number_R
2584,S12328,R98889,F000022561
2586,S37595,S33566,F000022561
2588,S74086,S56033,F000022561
2590,T34799,T31357,F000022561
7747,T99170,T73434,F000039122
...,...,...,...
75977,W30694,W07892,H000951723
75980,W30695,W07893,H000951723
75982,W38197,T28230,H000951723
75983,W38198,T94285,H000951723


In [191]:
#Create Batch Table for finished goods with linked production orders

#Keep Only 
df_batch = df_mm[df_mm['Part Type Code'].str.contains("FERT")==True]

#remove production orders values equal 'Not Set'
df_batch = df_batch[df_batch['Prod Order_R'].str.contains("Not Set")==False]

#Get unique Batch Numbers
df_batch = df_batch[['Batch Number_R','Prod Order_R','Material Number_R']].drop_duplicates(subset=['Batch Number_R'])
#Assumed each Batch Number_R has one Prod Order_R. There are 5 exceptions which have been deduped

df_batch



,Batch Number_R,Prod Order_R,Material Number_R
301,S27081,S27081,F000013151
305,S27082,S27082,F000013151
307,S27083,S27083,F000013151
312,S75194,S75194,F000013151
313,S69248,S69248,F000013151
...,...,...,...
64937,W56034,W56034,F000675699
65289,S57043,S57043,F000675750
65291,S791113,S791113,F000675750
65293,T35345,T35345,F000675750


In [192]:
link_table1 = pd.merge(df_batch, df_prod_ord_link, left_on='Prod Order_R', right_on='First Level Prod Ord',how='inner')

# Get indexes where Prod Order_R = Batch Number_R
index_batch_secpo = link_table1[link_table1['Linked Second Level Prod Ord'] == link_table1['Batch Number_R']].index
# Delete these row indexes from dataFrame
link_table1 = link_table1.drop(index_batch_secpo)
#Need to do this to stop circular looping for Production Order Linking

#Select relevant columns
link_table1 = link_table1[['Prod Order_R','Linked Second Level Prod Ord','Material Number_R_y']]

#Rename Columns
link_table1 = link_table1.rename(columns={"Prod Order_R": "First Level Prod Ord", "Linked Second Level Prod Ord": "Second Level Prod Ord", "Material Number_R_y": "Material Number - 1st level PO input" })
link_table1

,First Level Prod Ord,Second Level Prod Ord,Material Number - 1st level PO input
0,S27081,R897117,H000953195
1,S27082,R897118,H000953195
2,S27083,S28294,H000953195
3,S75194,S65997,H000953195
4,S69248,S35338,H000953195
...,...,...,...
2129,W56034,T49656,H000941018
2130,S57043,S37295,H000948883
2131,S791113,S70667,H000948883
2132,T35345,S76326,H000948883


In [193]:
link_table2 = pd.merge(link_table1, df_prod_ord_link, left_on='Second Level Prod Ord', right_on='First Level Prod Ord',how='inner')

#Select relevant columns
link_table2 = link_table2[['Second Level Prod Ord','Linked Second Level Prod Ord','Material Number_R']]

#Rename Columns
link_table2 = link_table2.rename(columns={"Linked Second Level Prod Ord": "Third Level Prod Ord", "Material Number_R": "Material Number - 2nd level PO input" })
link_table2

,Second Level Prod Ord,Third Level Prod Ord,Material Number - 2nd level PO input
0,S35338,R524111,H000133737
1,T27233,S27380,H000133737
2,T27234,S27381,H000133737
3,T27236,S27383,H000133737
4,T27235,S27382,H000133737
...,...,...,...
1728,T17054,S17166,H000133737
1729,T64263,S60638,H000133737
1730,T64267,S76136,H000133737
1731,W38885,T24124,H000133737


In [194]:
link_table3 = pd.merge(link_table2, df_prod_ord_link, left_on='Third Level Prod Ord', right_on='First Level Prod Ord',how='inner')

#Select relevant columns
link_table3 = link_table3[['Third Level Prod Ord','Linked Second Level Prod Ord','Material Number_R']]

#Rename Columns
link_table3 = link_table3.rename(columns={"Linked Second Level Prod Ord": "Fourth Level Prod Ord", "Material Number_R": "Material Number - 3rd level PO input" })
link_table3

,Third Level Prod Ord,Fourth Level Prod Ord,Material Number - 3rd level PO input
0,S76325,R89566,H000133737
1,S76325,R89566,H000133737
2,T44968,S76139,H000133737
3,W17123,T35288,H000133737
4,S32150,R524109,H000133737
...,...,...,...
172,S54852,R73173,H000133737
173,T70425,S76145,H000133737
174,S659105,R73177,H000133737
175,T780110,T07044,H000133737


In [195]:
#Add production Start and End Date to each level of Production Orders

#First Level
df_batch_dates = pd.merge(df_batch, df_prod_start_end, left_on='Prod Order_R', right_on='Prod Order_R',how='left')
df_batch_dates = df_batch_dates.rename(columns={"Posting Date min": "First Level Prod Start", "Posting Date max": "First Level Prod End" })
df_batch_dates = df_batch_dates.drop(['Posting Date count'], axis=1)
df_batch_dates['First Level Prod LT'] = (df_batch_dates['First Level Prod End'] - df_batch_dates['First Level Prod Start']).dt.days
df_batch_dates

,Batch Number_R,Prod Order_R,Material Number_R,First Level Prod Start,First Level Prod End,First Level Prod LT
0,S27081,S27081,F000013151,2017-02-03,2017-02-10,7
1,S27082,S27082,F000013151,2017-02-07,2017-02-13,6
2,S27083,S27083,F000013151,2017-02-08,2017-02-13,5
3,S75194,S75194,F000013151,2017-05-15,2017-05-17,2
4,S69248,S69248,F000013151,2017-05-16,2017-05-19,3
...,...,...,...,...,...,...
2251,W56034,W56034,F000675699,2018-02-01,2018-02-05,4
2252,S57043,S57043,F000675750,2017-02-20,2017-02-27,7
2253,S791113,S791113,F000675750,2017-04-27,2017-05-02,5
2254,T35345,T35345,F000675750,2017-06-29,2017-07-05,6


In [196]:
#Add production Start and End Date to each level of Production Orders

#Second Level
link_table1_dates = pd.merge(link_table1, df_prod_start_end, left_on='Second Level Prod Ord', right_on='Prod Order_R',how='left')
link_table1_dates = link_table1_dates.rename(columns={"Posting Date min": "Second Level Prod Start", "Posting Date max": "Second Level Prod End" })
link_table1_dates = link_table1_dates.drop(['Posting Date count', 'Prod Order_R'], axis=1)
link_table1_dates['Second Level Prod LT'] = (link_table1_dates['Second Level Prod End'] - link_table1_dates['Second Level Prod Start']).dt.days
link_table1_dates

,First Level Prod Ord,Second Level Prod Ord,Material Number - 1st level PO input,Second Level Prod Start,Second Level Prod End,Second Level Prod LT
0,S27081,R897117,H000953195,NaT,NaT,NaN
1,S27082,R897118,H000953195,NaT,NaT,NaN
2,S27083,S28294,H000953195,2017-01-13,2017-01-13,0.0
3,S75194,S65997,H000953195,2017-04-03,2017-04-20,17.0
4,S69248,S35338,H000953195,2017-01-12,2017-01-30,18.0
...,...,...,...,...,...,...
2109,W56034,T49656,H000941018,2017-09-11,2017-09-22,11.0
2110,S57043,S37295,H000948883,2017-01-16,2017-01-31,15.0
2111,S791113,S70667,H000948883,2017-03-23,2017-04-20,28.0
2112,T35345,S76326,H000948883,2017-04-25,2017-05-10,15.0


In [197]:
#Add production Start and End Date to each level of Production Orders

#Third Level
link_table2_dates = pd.merge(link_table2, df_prod_start_end, left_on='Third Level Prod Ord', right_on='Prod Order_R',how='left')
link_table2_dates = link_table2_dates.rename(columns={"Posting Date min": "Third Level Prod Start", "Posting Date max": "Third Level Prod End" })
link_table2_dates = link_table2_dates.drop(['Posting Date count', 'Prod Order_R'], axis=1)
link_table2_dates['Third Level Prod LT'] = (link_table2_dates['Third Level Prod End'] - link_table2_dates['Third Level Prod Start']).dt.days
link_table2_dates

,Second Level Prod Ord,Third Level Prod Ord,Material Number - 2nd level PO input,Third Level Prod Start,Third Level Prod End,Third Level Prod LT
0,S35338,R524111,H000133737,NaT,NaT,NaN
1,T27233,S27380,H000133737,2017-02-03,2017-02-03,0.0
2,T27234,S27381,H000133737,2017-02-03,2017-02-03,0.0
3,T27236,S27383,H000133737,2017-02-03,2017-02-03,0.0
4,T27235,S27382,H000133737,2017-02-03,2017-02-03,0.0
...,...,...,...,...,...,...
1728,T17054,S17166,H000133737,2017-02-16,2017-02-16,0.0
1729,T64263,S60638,H000133737,2017-03-23,2017-06-16,85.0
1730,T64267,S76136,H000133737,2017-03-25,2017-06-16,83.0
1731,W38885,T24124,H000133737,2017-06-27,2017-10-23,118.0


In [198]:
#Add production Start and End Date to each level of Production Orders

#Fourth Level
link_table3_dates = pd.merge(link_table3, df_prod_start_end, left_on='Fourth Level Prod Ord', right_on='Prod Order_R',how='left')
link_table3_dates = link_table3_dates.rename(columns={"Posting Date min": "Fourth Level Prod Start", "Posting Date max": "Fourth Level Prod End" })
link_table3_dates = link_table3_dates.drop(['Posting Date count', 'Prod Order_R'], axis=1)
link_table3_dates['Fourth Level Prod LT'] = (link_table3_dates['Fourth Level Prod End'] - link_table3_dates['Fourth Level Prod Start']).dt.days
link_table3_dates

,Third Level Prod Ord,Fourth Level Prod Ord,Material Number - 3rd level PO input,Fourth Level Prod Start,Fourth Level Prod End,Fourth Level Prod LT
0,S76325,R89566,H000133737,NaT,NaT,NaN
1,S76325,R89566,H000133737,NaT,NaT,NaN
2,T44968,S76139,H000133737,2017-04-01,2017-06-22,82.0
3,W17123,T35288,H000133737,2017-07-13,2017-10-04,83.0
4,S32150,R524109,H000133737,NaT,NaT,NaN
...,...,...,...,...,...,...
172,S54852,R73173,H000133737,NaT,NaT,NaN
173,T70425,S76145,H000133737,2017-04-09,2017-06-23,75.0
174,S659105,R73177,H000133737,NaT,NaT,NaN
175,T780110,T07044,H000133737,2017-06-02,2017-08-03,62.0


In [199]:
#Join all tables

df_batch_dates_comb1 = pd.merge(df_batch_dates, link_table1_dates, left_on='Prod Order_R', right_on='First Level Prod Ord',how='left')
df_batch_dates_comb1 = df_batch_dates_comb1.drop(['First Level Prod Ord'], axis=1)
df_batch_dates_comb1 = df_batch_dates_comb1.rename(columns={"Prod Order_R": "First Level Prod Ord", 
                                                          "Material Number_R": "Material Number - 1st level PO output"})
df_batch_dates_comb1 = df_batch_dates_comb1[['Batch Number_R', 'First Level Prod Ord', 'Material Number - 1st level PO output',
                                           'First Level Prod Start', 'First Level Prod End','First Level Prod LT','Second Level Prod Ord', 
                                           'Material Number - 1st level PO input','Second Level Prod Start', 
                                           'Second Level Prod End', 'Second Level Prod LT' ]]
df_batch_dates_comb1

,Batch Number_R,First Level Prod Ord,Material Number - 1st level PO output,First Level Prod Start,First Level Prod End,First Level Prod LT,Second Level Prod Ord,Material Number - 1st level PO input,Second Level Prod Start,Second Level Prod End,Second Level Prod LT
0,S27081,S27081,F000013151,2017-02-03,2017-02-10,7,R897117,H000953195,NaT,NaT,NaN
1,S27082,S27082,F000013151,2017-02-07,2017-02-13,6,R897118,H000953195,NaT,NaT,NaN
2,S27083,S27083,F000013151,2017-02-08,2017-02-13,5,S28294,H000953195,2017-01-13,2017-01-13,0.0
3,S75194,S75194,F000013151,2017-05-15,2017-05-17,2,S65997,H000953195,2017-04-03,2017-04-20,17.0
4,S69248,S69248,F000013151,2017-05-16,2017-05-19,3,S35338,H000953195,2017-01-12,2017-01-30,18.0
...,...,...,...,...,...,...,...,...,...,...,...
2253,W56034,W56034,F000675699,2018-02-01,2018-02-05,4,T49656,H000941018,2017-09-11,2017-09-22,11.0
2254,S57043,S57043,F000675750,2017-02-20,2017-02-27,7,S37295,H000948883,2017-01-16,2017-01-31,15.0
2255,S791113,S791113,F000675750,2017-04-27,2017-05-02,5,S70667,H000948883,2017-03-23,2017-04-20,28.0
2256,T35345,T35345,F000675750,2017-06-29,2017-07-05,6,S76326,H000948883,2017-04-25,2017-05-10,15.0


In [200]:
#Join all tables

df_batch_dates_comb2 = pd.merge(df_batch_dates_comb1, link_table2_dates, left_on='Second Level Prod Ord', right_on='Second Level Prod Ord',how='left')
df_batch_dates_comb2 = df_batch_dates_comb2.drop_duplicates()
df_batch_dates_comb2

,Batch Number_R,First Level Prod Ord,Material Number - 1st level PO output,First Level Prod Start,First Level Prod End,First Level Prod LT,Second Level Prod Ord,Material Number - 1st level PO input,Second Level Prod Start,Second Level Prod End,Second Level Prod LT,Third Level Prod Ord,Material Number - 2nd level PO input,Third Level Prod Start,Third Level Prod End,Third Level Prod LT
0,S27081,S27081,F000013151,2017-02-03,2017-02-10,7,R897117,H000953195,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,S27082,S27082,F000013151,2017-02-07,2017-02-13,6,R897118,H000953195,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,S27083,S27083,F000013151,2017-02-08,2017-02-13,5,S28294,H000953195,2017-01-13,2017-01-13,0.0,NaN,NaN,NaT,NaT,NaN
3,S75194,S75194,F000013151,2017-05-15,2017-05-17,2,S65997,H000953195,2017-04-03,2017-04-20,17.0,NaN,NaN,NaT,NaT,NaN
4,S69248,S69248,F000013151,2017-05-16,2017-05-19,3,S35338,H000953195,2017-01-12,2017-01-30,18.0,R524111,H000133737,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8589,W56034,W56034,F000675699,2018-02-01,2018-02-05,4,T49656,H000941018,2017-09-11,2017-09-22,11.0,S76144,H000133737,2017-04-07,2017-06-22,76.0
8592,S57043,S57043,F000675750,2017-02-20,2017-02-27,7,S37295,H000948883,2017-01-16,2017-01-31,15.0,R524112,H000133737,NaT,NaT,NaN
8598,S791113,S791113,F000675750,2017-04-27,2017-05-02,5,S70667,H000948883,2017-03-23,2017-04-20,28.0,R89564,H000133737,NaT,NaT,NaN
8606,T35345,T35345,F000675750,2017-06-29,2017-07-05,6,S76326,H000948883,2017-04-25,2017-05-10,15.0,S05395,H000133737,2017-01-04,2017-01-04,0.0


In [201]:
#Join all tables

df_batch_dates_comb3 = pd.merge(df_batch_dates_comb2, link_table3_dates, left_on='Third Level Prod Ord', right_on='Third Level Prod Ord',how='left')
df_batch_dates_comb3 = df_batch_dates_comb3.drop_duplicates()
df_batch_dates_comb3

,Batch Number_R,First Level Prod Ord,Material Number - 1st level PO output,First Level Prod Start,First Level Prod End,First Level Prod LT,Second Level Prod Ord,Material Number - 1st level PO input,Second Level Prod Start,Second Level Prod End,...,Third Level Prod Ord,Material Number - 2nd level PO input,Third Level Prod Start,Third Level Prod End,Third Level Prod LT,Fourth Level Prod Ord,Material Number - 3rd level PO input,Fourth Level Prod Start,Fourth Level Prod End,Fourth Level Prod LT
0,S27081,S27081,F000013151,2017-02-03,2017-02-10,7,R897117,H000953195,NaT,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
1,S27082,S27082,F000013151,2017-02-07,2017-02-13,6,R897118,H000953195,NaT,NaT,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2,S27083,S27083,F000013151,2017-02-08,2017-02-13,5,S28294,H000953195,2017-01-13,2017-01-13,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
3,S75194,S75194,F000013151,2017-05-15,2017-05-17,2,S65997,H000953195,2017-04-03,2017-04-20,...,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
4,S69248,S69248,F000013151,2017-05-16,2017-05-19,3,S35338,H000953195,2017-01-12,2017-01-30,...,R524111,H000133737,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2301,W56034,W56034,F000675699,2018-02-01,2018-02-05,4,T49656,H000941018,2017-09-11,2017-09-22,...,S76144,H000133737,2017-04-07,2017-06-22,76.0,NaN,NaN,NaT,NaT,NaN
2302,S57043,S57043,F000675750,2017-02-20,2017-02-27,7,S37295,H000948883,2017-01-16,2017-01-31,...,R524112,H000133737,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2303,S791113,S791113,F000675750,2017-04-27,2017-05-02,5,S70667,H000948883,2017-03-23,2017-04-20,...,R89564,H000133737,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN
2304,T35345,T35345,F000675750,2017-06-29,2017-07-05,6,S76326,H000948883,2017-04-25,2017-05-10,...,S05395,H000133737,2017-01-04,2017-01-04,0.0,NaN,NaN,NaT,NaT,NaN


In [202]:
#Create table for Second Level Prod LT to fill in NA
df_batch_dates_comb_avg1 = df_batch_dates_comb3[['Material Number - 1st level PO input', 'Second Level Prod LT']]
df_batch_dates_comb_avg1 = df_batch_dates_comb_avg1.dropna()
df_batch_dates_comb_avg1 = df_batch_dates_comb_avg1.groupby(['Material Number - 1st level PO input']).mean()
df_batch_dates_comb_avg1 = df_batch_dates_comb_avg1.round({'Second Level Prod LT': 0})

#Fill in empty 'Material Number - 1st level PO input'
df_batch_dates_comb_na1 = df_batch_dates_comb3[['Material Number - 1st level PO output', 'Material Number - 1st level PO input']]
df_batch_dates_comb_na1 = df_batch_dates_comb_na1.dropna()
df_batch_dates_comb_na1 = df_batch_dates_comb_na1.drop_duplicates(subset=['Material Number - 1st level PO output'])

df_batch_dates_comb4 = pd.merge(df_batch_dates_comb3, df_batch_dates_comb_na1, left_on='Material Number - 1st level PO output', right_on='Material Number - 1st level PO output',how='left')
df_batch_dates_comb4 = df_batch_dates_comb4.drop_duplicates()

df_batch_dates_comb4['Material Number - 1st level PO input_x'] = df_batch_dates_comb4['Material Number - 1st level PO input_x'].fillna(df_batch_dates_comb4.pop('Material Number - 1st level PO input_y'))
df_batch_dates_comb4 = df_batch_dates_comb4.rename(columns={"Material Number - 1st level PO input_x": "Material Number - 1st level PO input"})

df_batch_dates_comb5 = pd.merge(df_batch_dates_comb4, df_batch_dates_comb_avg1, left_on='Material Number - 1st level PO input', right_on='Material Number - 1st level PO input',how='left')

df_batch_dates_comb5['Second Level Prod LT_x'] = df_batch_dates_comb5['Second Level Prod LT_x'].fillna(df_batch_dates_comb5.pop('Second Level Prod LT_y'))
#df_batch_dates_comb5 = df_batch_dates_comb5.drop(['Second Level Prod LT_y'], axis=1)
df_batch_dates_comb5 = df_batch_dates_comb5.rename(columns={"Second Level Prod LT_x": "Second Level Prod LT"})
df_batch_dates_comb5 = df_batch_dates_comb5.dropna(subset=['Second Level Prod LT'])
df_batch_dates_comb5.to_excel("output.xlsx")


In [203]:
#Create table for Third Level Prod LT to fill in NA
df_batch_dates_comb_avg2 = df_batch_dates_comb5[['Material Number - 2nd level PO input', 'Third Level Prod LT']]
df_batch_dates_comb_avg2 = df_batch_dates_comb_avg2.dropna()
df_batch_dates_comb_avg2 = df_batch_dates_comb_avg2.groupby(['Material Number - 2nd level PO input']).mean()
df_batch_dates_comb_avg2 = df_batch_dates_comb_avg2.round({'Third Level Prod LT': 0})

#Fill in empty 'Material Number - 2nd level PO input'
df_batch_dates_comb_na2 = df_batch_dates_comb5[['Material Number - 1st level PO input', 'Material Number - 2nd level PO input']]
df_batch_dates_comb_na2 = df_batch_dates_comb_na2.dropna()
df_batch_dates_comb_na2 = df_batch_dates_comb_na2.drop_duplicates(subset=['Material Number - 1st level PO input'])

df_batch_dates_comb6 = pd.merge(df_batch_dates_comb5, df_batch_dates_comb_na2, left_on='Material Number - 1st level PO input', right_on='Material Number - 1st level PO input',how='left')
df_batch_dates_comb6 = df_batch_dates_comb6.drop_duplicates()

df_batch_dates_comb6['Material Number - 2nd level PO input_x'] = df_batch_dates_comb6['Material Number - 2nd level PO input_x'].fillna(df_batch_dates_comb6.pop('Material Number - 2nd level PO input_y'))
df_batch_dates_comb6 = df_batch_dates_comb6.rename(columns={"Material Number - 2nd level PO input_x": "Material Number - 2nd level PO input"})

df_batch_dates_comb7 = pd.merge(df_batch_dates_comb6, df_batch_dates_comb_avg2, left_on='Material Number - 2nd level PO input', right_on='Material Number - 2nd level PO input',how='left')

df_batch_dates_comb7['Third Level Prod LT_x'] = df_batch_dates_comb7['Third Level Prod LT_x'].fillna(df_batch_dates_comb7.pop('Third Level Prod LT_y'))
df_batch_dates_comb7 = df_batch_dates_comb7.rename(columns={"Third Level Prod LT_x": "Third Level Prod LT"})
#df_batch_dates_comb7 = df_batch_dates_comb7.dropna(subset=['Third Level Prod LT'])
df_batch_dates_comb7

,Batch Number_R,First Level Prod Ord,Material Number - 1st level PO output,First Level Prod Start,First Level Prod End,First Level Prod LT,Second Level Prod Ord,Material Number - 1st level PO input,Second Level Prod Start,Second Level Prod End,...,Third Level Prod Ord,Material Number - 2nd level PO input,Third Level Prod Start,Third Level Prod End,Third Level Prod LT,Fourth Level Prod Ord,Material Number - 3rd level PO input,Fourth Level Prod Start,Fourth Level Prod End,Fourth Level Prod LT
0,S27081,S27081,F000013151,2017-02-03,2017-02-10,7,R897117,H000953195,NaT,NaT,...,NaN,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
1,S27082,S27082,F000013151,2017-02-07,2017-02-13,6,R897118,H000953195,NaT,NaT,...,NaN,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
2,S27083,S27083,F000013151,2017-02-08,2017-02-13,5,S28294,H000953195,2017-01-13,2017-01-13,...,NaN,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
3,S75194,S75194,F000013151,2017-05-15,2017-05-17,2,S65997,H000953195,2017-04-03,2017-04-20,...,NaN,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
4,S69248,S69248,F000013151,2017-05-16,2017-05-19,3,S35338,H000953195,2017-01-12,2017-01-30,...,R524111,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,W56034,W56034,F000675699,2018-02-01,2018-02-05,4,T49656,H000941018,2017-09-11,2017-09-22,...,S76144,H000133737,2017-04-07,2017-06-22,76.0,NaN,NaN,NaT,NaT,NaN
2211,S57043,S57043,F000675750,2017-02-20,2017-02-27,7,S37295,H000948883,2017-01-16,2017-01-31,...,R524112,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
2212,S791113,S791113,F000675750,2017-04-27,2017-05-02,5,S70667,H000948883,2017-03-23,2017-04-20,...,R89564,H000133737,NaT,NaT,52.0,NaN,NaN,NaT,NaT,NaN
2213,T35345,T35345,F000675750,2017-06-29,2017-07-05,6,S76326,H000948883,2017-04-25,2017-05-10,...,S05395,H000133737,2017-01-04,2017-01-04,0.0,NaN,NaN,NaT,NaT,NaN


In [212]:
#Create table for Fourth Level Prod LT to fill in NA
df_batch_dates_comb_avg3 = df_batch_dates_comb7[['Material Number - 3rd level PO input', 'Fourth Level Prod LT']]
df_batch_dates_comb_avg3 = df_batch_dates_comb_avg3.dropna()
df_batch_dates_comb_avg3 = df_batch_dates_comb_avg3.groupby(['Material Number - 3rd level PO input']).mean()
df_batch_dates_comb_avg3 = df_batch_dates_comb_avg3.round({'Fourth Level Prod LT': 0})

#Fill in empty 'Material Number - 3rd level PO input'
df_batch_dates_comb_na3 = df_batch_dates_comb7[['Material Number - 2nd level PO input', 'Material Number - 3rd level PO input']]
df_batch_dates_comb_na3 = df_batch_dates_comb_na3.dropna()
df_batch_dates_comb_na3 = df_batch_dates_comb_na3.drop_duplicates(subset=['Material Number - 2nd level PO input'])

df_batch_dates_comb8 = pd.merge(df_batch_dates_comb7, df_batch_dates_comb_na3, left_on='Material Number - 2nd level PO input', right_on='Material Number - 2nd level PO input',how='left')
df_batch_dates_comb8 = df_batch_dates_comb8.drop_duplicates()

df_batch_dates_comb8['Material Number - 3rd level PO input_x'] = df_batch_dates_comb8['Material Number - 3rd level PO input_x'].fillna(df_batch_dates_comb8.pop('Material Number - 3rd level PO input_y'))
df_batch_dates_comb8 = df_batch_dates_comb8.rename(columns={"Material Number - 3rd level PO input_x": "Material Number - 3rd level PO input"})

df_batch_dates_comb9 = pd.merge(df_batch_dates_comb8, df_batch_dates_comb_avg3, left_on='Material Number - 3rd level PO input', right_on='Material Number - 3rd level PO input',how='left')

df_batch_dates_comb9['Fourth Level Prod LT_x'] = df_batch_dates_comb9['Fourth Level Prod LT_x'].fillna(df_batch_dates_comb9.pop('Fourth Level Prod LT_y'))
df_batch_dates_comb9 = df_batch_dates_comb9.rename(columns={"Fourth Level Prod LT_x": "Fourth Level Prod LT"})
df_batch_dates_comb9

In [220]:
df_batch_dates_comb9['E2E Prod LT'] = df_batch_dates_comb9.fillna(0)['First Level Prod LT'] + df_batch_dates_comb9.fillna(0)['Second Level Prod LT'] + df_batch_dates_comb9.fillna(0)['Third Level Prod LT'] + df_batch_dates_comb9.fillna(0)['Fourth Level Prod LT']


In [221]:
df_batch_dates_comb9.to_excel("output.xlsx")